# Data Acquisition Lesson

In [4]:
import requests
import pandas as pd

### Making HTTP Requests

In [3]:
# https://aphorisms.glitch.me returns a random quotation
response = requests.get('http://aphorisms.glitch.me/')
response

<Response [200]>

In [5]:
#.ok: a boolean that indicates that the response was successful (the server sent back a 200 response code)
response.ok

True

In [6]:
#.status_code: a number indicating the HTTP response status code 
response.status_code

200

In [7]:
#.text: the raw response text
response.text

'{"quote":"Everything we are taught is an approximation of some reality out there\u2029","author":"Ravinder Singh"}'

### Example JSON API

For an example of a JSON api, we'll interact with the a quote generator.

In [8]:
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)

{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"https://swapi.dev/api/planets/2/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"https://swapi.dev/api/people/5/"}


Here we see that the repsonse we got back contains a JSON object (we could also verify this by visiting the URL in a web browser).

Since the response is JSON, we can use the .json method on the response object to get a data structure we can work with:

In [9]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'https://swapi.dev/api/planets/2/',
 'films': ['https://swapi.dev/api/films/1/',
  'https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['https://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'https://swapi.dev/api/people/5/'}

Now we have a dictionary that we can work with.

Let's now take a look at another api. We'll start by looking at just the base URL: 

In [10]:
base_url = 'https://python.zgulde.net'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



This API provides some documentation, so let's make a request so that we can take a look at it.

In [11]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



Based on this, let's take a look at the items. We'll make our request, and explore the shape of the response that we get back.

In [12]:
response = requests.get('https://python.zgulde.net/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [13]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [14]:
current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

current_page: 1
max_page: 3
next_page: /api/v1/items?page=2


Here the response has some built-in properties that tell us how to get to subsequent pages.

Once we've drilled down into the data structure, we'll find that the entire response is a sort of wrapper around the items property:

In [15]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

We can turn this data into a pandas dataframe:

In [16]:
import pandas as pd

df = pd.DataFrame(data['payload']['items'])
df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


Now that we've gotten the data from the first page, we can extract the data from the next page (as indicated by the API's response), and add it onto our dataframe:

In [17]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

current_page: 2
max_page: 3
next_page: /api/v1/items?page=3


We'll repeat the process one more time:

In [18]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

current_page = data['payload']['page']
max_page = data['payload']['max_page']
next_page = data['payload']['next_page']

print(f'current_page: {current_page}')
print(f'max_page: {max_page}')
print(f'next_page: {next_page}')

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

current_page: 3
max_page: 3
next_page: None


Now that the API says that the next_page is None, we'll stop making requests, and assume that we have all of the items data.

In [19]:
df.shape

(50, 8)

#### Exercises

Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [50]:
#extract multiple pages while loop

#base 
domain = 'https://python.zgulde.net'

#where url is leading
endpoint = '/api/v1/items'

#create empty list

items = []

while True:
    #create url variable
    url = domain + endpoint
    #get url
    response = requests.get(url)
    #create a dict from response
    data = response.json()
    #use print response
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
    #add each page to empty list
    items.extend(data['payload']['items'])
    #update endpoint to cycle through pages
    endpoint = data['payload']['next_page']
    #once there are no empty pages, break
    if endpoint is None:
        break

Getting page 3 of 3: https://python.zgulde.net/api/v1/items?page=3

In [53]:
#turn into df
items = pd.DataFrame(items)

In [54]:
items

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


In [56]:
#extract multiple pages for loop

base_url = 'https://python.zgulde.net/api/v1/items?page='
items2 = []

#make first request
url = base_url + str(1)
response = requests.get(url)
data = response.json()
max_page = data['payload']['max_page']
items2.extend(data['payload']['items'])

#bc already req'd first page, start loop at 2
#add 1 to max_page bc range() is exclusive of the endpoint

page_range = range(2, max_page + 1)

for page in page_range:
    url = base_url + str(page)
    print(f'\Fetching page {page}/{max_page} {url}', end = '')
    response = requests.get(url)
    data = response.json()
    items2.extend(data['payload']['items'])

\Fetching page 2/3 https://python.zgulde.net/api/v1/items?page=2\Fetching page 3/3 https://python.zgulde.net/api/v1/items?page=3

In [59]:
items2 = pd.DataFrame(items2)
items2.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [64]:
#response = requests.get('https://python.zgulde.net/api/v1/stores')
#data = response.json()

#data.keys() #dict keys: 'payload', 'status'
#data['payload']


#extract multiple pages while loop

#base 
domain = 'https://python.zgulde.net'

#where url is leading
endpoint = '/api/v1/stores'

#create empty list

stores = []

while True:
    #create url variable
    url = domain + endpoint
    #get url
    response = requests.get(url)
    #create a dict from response
    data = response.json()
    #use print response
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
    #add each page to empty list
    stores.extend(data['payload']['stores'])
    #update endpoint to cycle through pages
    endpoint = data['payload']['next_page']
    #once there are no empty pages, break
    if endpoint is None:
        break

Getting page 1 of 1: https://python.zgulde.net/api/v1/stores

In [67]:
#stores

#create df
stores = pd.DataFrame(stores)
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [68]:
#get url
response = requests.get('https://python.zgulde.net/api/v1/sales')

#use .json() to turn into dictionary
data = response.json()

#get keys
data.keys()

#data

dict_keys(['payload', 'status'])

In [70]:
#base 
domain = 'https://python.zgulde.net'

#where url is leading
endpoint = '/api/v1/sales'

#create empty list

sales = []

while True:
    #create url variable
    url = domain + endpoint
    #get url
    response = requests.get(url)
    #create a dict from response
    data = response.json()
    #use print response
    print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
    #add each page to empty list
    sales.extend(data['payload']['sales'])
    #update endpoint to cycle through pages
    endpoint = data['payload']['next_page']
    #once there are no empty pages, break
    if endpoint is None:
        break

Getting page 183 of 183: https://python.zgulde.net/api/v1/sales?page=183

In [71]:
sales = pd.DataFrame(sales)
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


Save the data in your files to local csv files so that it will be faster to access in the future.
Combine the data from your three separate dataframes into one large dataframe.

In [72]:
items.to_csv('items.csv', index = False)
stores.to_csv('stores.csv', index = False)
sales.to_csv('sales.csv', index = False)

In [78]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [79]:
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [84]:
#get unique item names in sales
sales.item.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [85]:
#get unique item_id names in items
items.item_id.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [87]:
items.shape

(50, 6)

In [86]:
#is the number of unique items the same as df rows?
items.item_id.nunique() == items.shape[0]

True

In [88]:
sales.store.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [89]:
stores.store_id.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [90]:
#since item in sales and item_id in items have the same values, 
# and sales store and stores store_id are the same rename in sales df

sales = sales.rename(columns = {'item': 'item_id', 'store': 'store_id'})

In [92]:
sales.head()

,item_id,sale_amount,sale_date,sale_id,store_id
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [93]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [95]:
#create df with sales and items, merge left on item_id
df = pd.merge(sales, items, how = 'left', on = 'item_id')

df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [96]:
#df: merge left with stores on store_id

df = pd.merge(df, stores, how = 'left', on = 'store_id')

df.head()

,item_id,sale_amount,sale_date,sale_id,store_id,item_brand,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_state,store_zipcode
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


In [97]:
df.shape

(913000, 14)

---

Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [76]:
opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

In [77]:
opsd.to_csv('opsd.csv', index = False)

In [98]:
opsd.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [99]:
opsd.isna().sum()

Date              0
Consumption       0
Wind           1463
Solar          2195
Wind+Solar     2196
dtype: int64

In [100]:
opsd.shape

(4383, 5)

In [ ]:
def get_store_data_from_api():    
    #extract multiple pages while loop
    #base 
    domain = 'https://python.zgulde.net'

    #where url is leading
    endpoint = '/api/v1/stores'

    #create empty list

    stores = []

    while True:
        #create url variable
        url = domain + endpoint
        #get url
        response = requests.get(url)
        #create a dict from response
        data = response.json()
        #use print response
        print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
        #add each page to empty list
        stores.extend(data['payload']['stores'])
        #update endpoint to cycle through pages
        endpoint = data['payload']['next_page']
        #once there are no empty pages, break
        if endpoint is None:
            break

    return stores

In [ ]:
def get_items_data_from_api():
    #extract multiple pages while loop

    #base 
    domain = 'https://python.zgulde.net'

    #where url is leading
    endpoint = '/api/v1/items'

    #create empty list

    items = []

    while True:
        #create url variable
        url = domain + endpoint
        #get url
        response = requests.get(url)
        #create a dict from response
        data = response.json()
        #use print response
        print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
        #add each page to empty list
        items.extend(data['payload']['items'])
        #update endpoint to cycle through pages
        endpoint = data['payload']['next_page']
        #once there are no empty pages, break
        if endpoint is None:
            break

    return items

In [ ]:
def get_sales_data_from_api():
    #base 
    domain = 'https://python.zgulde.net'

    #where url is leading
    endpoint = '/api/v1/sales'

    #create empty list

    sales = []

    while True:
        #create url variable
        url = domain + endpoint
        #get url
        response = requests.get(url)
        #create a dict from response
        data = response.json()
        #use print response
        print(f'\rGetting page {data["payload"]["page"]} of {data["payload"]["max_page"]}: {url}', end = '')
        #add each page to empty list
        sales.extend(data['payload']['sales'])
        #update endpoint to cycle through pages
        endpoint = data['payload']['next_page']
        #once there are no empty pages, break
        if endpoint is None:
            break

    return sales


In [ ]:
def get_stores_data():
    if os.path.exists('stores.csv'):
        return pd.read_csv('stores.csv')
    df = get_store_data_from_api()
    df.to_csv('stores.csv', index = False)
    return df


In [ ]:
def get_items_data():
    if os.path.exists('items.csv'):
        return pd.read_csv('items.csv')
    df = get_items_data_from_api()
    df.to_csv('items.csv', index = False)
    return df

In [ ]:
def get_sales_data():
    if os.path.exists('sales.csv'):
        return pd.read_csv('sales.csv')
    df = get_sales_data_from_api()
    df.to_csv('sales.csv', index = False)
    return df

In [ ]:
def get_store_item_demand_data():
    sales = get_sales_data()
    stores = get_stores_data()
    items = get_items_data()

    sales = sales.rename(columns = {'store': 'store_id', 'item': 'item_id'})
    df = pd.merge(sales, stores, how = 'left', on = 'store_id')
    df = pd.merge(df, items, how = 'left', on = 'item_id')

    return df

In [ ]:
def get_opsd_data():
    if os.path.exists('opsd.csv'):
        return pd.read_csv('opsd.csv')
    df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
    df.to_csv('opsd.csv', index = False)
    return df